In [ ]:


# Import the libraries we need to use in this lab

# Using the following line code to install the torchvision library
#!pip install -y torchvision

#!pip install torchvision==0.9.1 torch==1.8.1 
import torch 
import torch.nn as nn
import torch.optim as optim
import os
import torchvision
from torchvision.io import read_image
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pylab as plt
import numpy as np
from PIL import Image

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a common size
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean values
        std=[0.229, 0.224, 0.225]    # Normalize with ImageNet standard deviation values
    ),
])


In [ ]:
path_to_root = '' # pictures from either images or spliced video frames


In [ ]:
dataset = torchvision.datasets.ImageFolder(root=path_to_root, transform=transform)

In [ ]:
# print some stuff here to check what we read in
print(len(dataset))
print(dataset[255][1])
print(dataset[0][0])
print(dataset)

In [ ]:
def imshow(image, title=None):
    image = image.numpy().transpose((1, 2, 0))
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = std * image + mean
    image = np.clip(image, 0, 1)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.axis('off')

In [ ]:
image_tensor, label = dataset[200]  # Replace '0' with the desired index
imshow(image_tensor, title=f'Class: {dataset.classes[label]}')
plt.show()
#class_list = ['blank', 'paper', 'rock', 'scissors']

In [ ]:
def show_data(data_sample):
    plt.imshow(data_sample[0][0])
    plt.title('y = '+ str(data_sample[1]))
    
show_data(dataset[200])

In [ ]:
batch_size = 64
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
model = torchvision.models.resnet18(pretrained=True)
num_classes = len(dataset.classes)  # Number of classes in your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify the classifier


In [ ]:
print(num_classes)

In [ ]:
criterion = nn.CrossEntropyLoss()  # Classification loss
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [ ]:
num_epochs = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss / len(dataloader)}')


In [ ]:
# After training your model
torch.save(model.state_dict(), 'model_checkpoint.pth')
